In [1]:
import numpy as np
import pandas as pd
import random
#other
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import string
punctuations = string.punctuation
from tqdm import tqdm

### Load les anciens trucs

#### Truc extrait et processer de Spacy NER 

In [2]:
all_locs_ns_str=np.load('../../data/locs_insee_str.npy', allow_pickle=True)

In [5]:
all_locs_ns_str[275]

'issy-les-moulineaux Issy-les-Moulineaux'

#### Extrait des tableaux insee

In [4]:
departements=pd.read_csv('../../data/departement2019.csv')
communes=pd.read_csv('../../data/communes-01012019.csv')
regions=pd.read_csv('../../data/region2019.csv')

In [5]:
regions_names=regions['nccenr'].to_numpy()
departements_names=departements['nccenr'].to_numpy()
communes_names=communes['nccenr'].to_numpy()

#### Permet de filtrer certains trucs ou on est pas sur

In [6]:
communes_prblm = list(np.load('../../data/communes_prblm.npy', allow_pickle=True))

#### Resultats avec l'ancien algo

In [7]:
locs_pred=np.array(pd.read_csv('../../data/Catalogue_locs.csv')['pred locs'])

In [8]:
locs_pred

array(['PARIS', 'CHAMBERY', 'CHAMBERY', ..., 'ARLES', 'ARLES', 'ARLES'],
      dtype=object)

### Algo débile

In [9]:
resultats=[]
count1=0
for loc in tqdm(all_locs_ns_str):
    found=0
    for ref in list(communes_names)+list(departements_names)+list(regions_names):
        if ref in loc.split():
            resultats.append([loc,ref])
            found=1
            count1+=1
            break
    if found==0:
        resultats.append([loc,' '])

  4%|▍         | 1665/42837 [00:11<04:40, 146.56it/s]


KeyboardInterrupt: 

In [75]:
count1

23395

In [76]:
count

11965

In [44]:
len(resultats)

42837

In [45]:
resultats_brut=np.array(resultats).transpose()[1]

### On merge les deux (on prend l'intersection)

In [46]:
len(all_locs_ns_str)

42837

In [59]:
resultats_brut[1025]

'Hauts-de-Seine'

In [63]:
locs_pred[1025]==resultats_brut[1025].upper().replace('-',' ')

True

In [66]:
currated_list=[]
count=0
for i in range(len(locs_pred)):
    if locs_pred[i]==resultats_brut[i].upper().replace('-', ' '):
        currated_list.append(locs_pred[i])
        count+=1
    else:
        currated_list.append('not sure')

In [ ]:
currated_list[430:520]

In [69]:
count/len(resultats)

0.27931461120059764

##### Idée: je pourrais recuperer le score de différents fuzzy matching + donner un score pour le subword; entrainer un truc debile genre random forest sur une partie 'annotée' et le laisser 'faire le meilleur algo' en fonction des scores et des subwords pour avoir le max de précision -\_/- 

### On fait que sur les organisations et on extend

#### Partie débile

In [37]:
data=np.load('../../data/organizations_col.npy', allow_pickle=True)

In [38]:
data_single=list(dict.fromkeys(data))
data_single=[x for x in data_single if str(x) != 'nan']

In [39]:
org_locs=[]
count1=0
for loc in tqdm(data_single):
    found=0
    for ref in list(communes_names)+list(departements_names)+list(regions_names):
        if ref in loc.split():
            org_locs.append([loc,ref])
            found=1
            count1+=1
            break
    if found==0:
        org_locs.append([loc,' '])

100%|██████████| 1161/1161 [00:07<00:00, 148.08it/s]


In [40]:
org_locs

[['Mairie de Paris', 'Paris'],
 ['Grand Chambéry', 'Chambéry'],
 ['Collectivité de Corse', 'Corse'],
 ['Orléans Métropole', 'Orléans'],
 ['Mairie de Saint-Denis (La Réunion)', 'Saint-Denis'],
 ['Réseau  régional Inspection Contrôle Evaluation DR(D)JSCS', ' '],
 ['Mendionde', 'Mendionde'],
 ['Gamarthe', 'Gamarthe'],
 ['Île-de-France Mobilités', 'Île-de-France'],
 ['Ministère de la Culture', ' '],
 ['Grand Paris Seine Ouest', 'Paris'],
 ['OpenStreetMap', ' '],
 ["Syndicat Mixte des Transports en Commun de l'Agglomération Clermontoise",
  'Syndicat'],
 ["Ville d'Antibes", 'Ville'],
 ["Office national d'information sur les enseignements et les professions",
  ' '],
 ['Département de Loire-Atlantique', 'Loire-Atlantique'],
 ['Région Île-de-France', 'Île-de-France'],
 ['Région Centre-Val de Loire', 'Loire'],
 ["Département de l'Hérault", ' '],
 ['Conseil départemental de Haute-Garonne', 'Haute-Garonne'],
 ["Ministère de l'Enseignement supérieur, de la Recherche et de l'Innovation",
  ' '],
 

#### Load du notebook 'sophistiqué'

In [41]:
org_locs_alg=np.load('../../data/locs_org_alg1.npy', allow_pickle=True)

In [42]:
org_locs[:100]

[['Mairie de Paris', 'Paris'],
 ['Grand Chambéry', 'Chambéry'],
 ['Collectivité de Corse', 'Corse'],
 ['Orléans Métropole', 'Orléans'],
 ['Mairie de Saint-Denis (La Réunion)', 'Saint-Denis'],
 ['Réseau  régional Inspection Contrôle Evaluation DR(D)JSCS', ' '],
 ['Mendionde', 'Mendionde'],
 ['Gamarthe', 'Gamarthe'],
 ['Île-de-France Mobilités', 'Île-de-France'],
 ['Ministère de la Culture', ' '],
 ['Grand Paris Seine Ouest', 'Paris'],
 ['OpenStreetMap', ' '],
 ["Syndicat Mixte des Transports en Commun de l'Agglomération Clermontoise",
  'Syndicat'],
 ["Ville d'Antibes", 'Ville'],
 ["Office national d'information sur les enseignements et les professions",
  ' '],
 ['Département de Loire-Atlantique', 'Loire-Atlantique'],
 ['Région Île-de-France', 'Île-de-France'],
 ['Région Centre-Val de Loire', 'Loire'],
 ["Département de l'Hérault", ' '],
 ['Conseil départemental de Haute-Garonne', 'Haute-Garonne'],
 ["Ministère de l'Enseignement supérieur, de la Recherche et de l'Innovation",
  ' '],
 

In [43]:
df_loc_orgs=org_locs_alg[0]
for df in org_locs_alg[1:]:
    try:
        df_loc_orgs=pd.concat([df_loc_orgs, df], ignore_index=True, sort=False)
    except:
        df=pd.DataFrame([df], columns=['ncc'])
        df_loc_orgs=pd.concat([df_loc_orgs, df], ignore_index=True, sort=False)

In [68]:
org_locs_ref=np.array(org_locs).transpose()[0]

In [72]:
org_locs_simple=np.array(org_locs).transpose()[1]

In [69]:
df_loc_orgs['ref org']=org_locs_ref

In [70]:
df_loc_orgs

,typecom,com,reg,dep,arr,tncc,ncc,nccenr,libelle,can,comparent,cheflieu,ref org
0,COM,75056,11.0,75,751,0.0,PARIS,Paris,Paris,NaN,NaN,NaN,Mairie de Paris
1,COM,73065,84.0,73,732,0.0,CHAMBERY,Chambéry,Chambéry,7399,NaN,NaN,Grand Chambéry
2,NaN,NaN,94.0,NaN,NaN,0.0,CORSE,Corse,Corse,NaN,NaN,2A004,Collectivité de Corse
3,COM,45234,24.0,45,452,1.0,ORLEANS,Orléans,Orléans,4599,NaN,NaN,Orléans Métropole
4,NaN,NaN,4.0,974,NaN,0.0,LA REUNION,La Réunion,La Réunion,NaN,NaN,97411,Mairie de Saint-Denis (La Réunion)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,COM,46030,76.0,46,463,0.0,BIO,Bio,Bio,4610,NaN,NaN,Agence Bio
1157,NaN,NaN,93.0,06,NaN,4.0,ALPES MARITIMES,Alpes-Maritimes,Alpes-Maritimes,NaN,NaN,06088,Secrétariat général
1158,COM,94068,11.0,94,942,0.0,SAINT MAUR DES FOSSES,Saint-Maur-des-Fossés,Saint-Maur-des-Fossés,9496,NaN,NaN,Saint-Maur des Fossés
1159,COM,93077,11.0,93,932,0.0,VILLEMOMBLE,Villemomble,Villemomble,9321,NaN,NaN,Villemomble


In [ ]:
df_loc_org

In [71]:
pd.DataFrame(df_loc_orgs).to_csv('../../data/locs_org.csv')

#### Let's merge

In [66]:
org_locs_alg[25]['nccenr'].values[0]

'Saint-Claude'

In [73]:
org_locs_simple[25]

'Saint-Claude'

In [74]:
org_locs_simple[258]

' '

In [84]:
currated_list_orgs=[]
count=0
for i in range(len(org_locs_alg)):
    try:
        if org_locs_alg[i]['nccenr'].values[0]=='Syndicat':
            currated_list_orgs.append('not sure')
            count+=1
            continue
        elif org_locs_alg[i]['nccenr'].values[0]==org_locs_simple[i]:
            currated_list_orgs.append(org_locs_simple[i])
            count+=1
        else:
            currated_list_orgs.append('not sure')
    except:
        currated_list_orgs.append('not sure')
        count+=1

In [85]:
len(currated_list_orgs)

1161

In [86]:
count

460

In [87]:
currated_list_orgs

['Paris',
 'Chambéry',
 'Corse',
 'Orléans',
 'not sure',
 'not sure',
 'Mendionde',
 'Gamarthe',
 'Île-de-France',
 'not sure',
 'Paris',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'Loire-Atlantique',
 'Île-de-France',
 'not sure',
 'not sure',
 'Haute-Garonne',
 'not sure',
 'Strasbourg',
 'Doubs',
 'Nice',
 'not sure',
 'Saint-Claude',
 'not sure',
 'Nantes',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'Bourgogne-Franche-Comté',
 'not sure',
 'not sure',
 'not sure',
 'Toulouse',
 'not sure',
 'not sure',
 'Hauts-de-France',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'not sure',
 'Bretagne',
 'not sure',
 'not sure',
 'Bretagne',
 'Occitanie',
 'not sure',
 'not sure',
 'Hautes-Pyrénées',
 'Nantes',
 'not sure',
 'not sure',
 'Chalon',
 